This notebook is for purely supervised training

In [1]:
#%matplotlib inline
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import tensorflow as tf
import sys, os
from fnmatch import fnmatch
from keras.models import Sequential 
from keras.layers import Activation, Dense, Flatten,Conv1D, MaxPooling1D,Dropout,GlobalAveragePooling1D

from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import concatenate
import glob
import math
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

In [2]:
# to get all .pkl files in a base directory. It will go to every sub directory find the .pkl file

signals = [] # it will store all EDA signals of all subjects from "chest" device
root = "Dataset/WESAD/untitled folder/"
pattern = "*.pkl"
signal_name='EDA'

file_paths = glob.glob(root+pattern)
subs=[]
for path in  file_paths:
    print(path)
    with open(path, 'rb') as file:
        data = pickle.load(file, encoding='latin1')
    eda_signal= data['signal']['chest'][signal_name][:,0]
    label=data['label']
    numpy_data1=np.array([eda_signal,label])
    numpy_data1=numpy_data1.T
    df = pd.DataFrame(data=numpy_data1, columns=[signal_name,"Label"]) 
    df= df[df["Label"]==1].reset_index(drop=True)
    signals.append(df[signal_name].values)
            

In [3]:
subjects=[x.split('/')[-1].split('.')[0] for x in file_paths]

In [3]:
print(subjects)

In [5]:
import random
def delete_random_elems(input_list, n):
    to_delete = set(random.sample(range(len(input_list)), n))
    return [x for i,x in enumerate(input_list) if not i in to_delete]


signal_same_length = []

for signal in signals:
    if len(signal)==min([len(x) for x in signals]):
        signal_same_length.append(signal)
    else:
        signal_same_length.append(np.array(delete_random_elems(signal, len(signal)-min([len(x) for x in signals]))))


In [10]:

subjects = ['S5', 'S2', 'S3', 'S4', 'S17', 'S10', 'S11', 'S16', 'S8',
            'S6', 'S7', 'S9', 'S13', 'S14', 'S15']

for question in range(0,6):
    y=[]
    i=0
    for subject in subjects:
        with open('/Users/ullash/MastersThesis/Dataset/QuestLabels/'+subject+'.pickle', 'rb') as handle:
            b = pickle.load(handle)
            y.append(b['values']['Base']['stai'][question])
    y=pd.DataFrame(y)
    y[0]= y[0].map({1:0.25,2:0.5,3:0.75,4:1})
    y = list(y[0].values)
    X=signal_same_length
    
    print("Question: "+str(question))
    for subject in subjects:
        print(f'Subject ID: {subject}')
        output = evaluate_model(subject,X,y,i,question)
        evaluation_list.append(output)
        i=i+1
    

In [7]:
def build_data_with_forecasting(signal,window,output_length, overlap):
    X=[]
    y=[]
    start=0
    while (True):
        x1=signal[start:start+window]
        output=signal[start+window:start+window+output_length] # 10 points
        if (len(output)!=output_length):
            break
        X.append(x1)
        y.append(output)
        start=start+overlap
    return X

In [ ]:
from keras import metrics
def evaluate_model(trainX,trainy,testX,testy,subject,signal,question,data_point,step):
    trainX,trainy,testX,testy=np.array(trainX),np.array(trainy),np.array(testX),np.array(testy)
    
    verbose, epochs, batch_size = 0, 100, 1000
    model = tf.keras.models.Sequential()
    model.add(Conv1D(40, 4, activation='leaky_relu',padding="same", input_shape=(trainX.shape[1],1)))
    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))
    model.add(Conv1D(18, 4,padding="same", activation='leaky_relu'))
    model.add(Conv1D(30, 2,padding="same", activation='leaky_relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(70, activation='leaky_relu'))
    model.add(Dense(30, activation='leaky_relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(loss="mean_squared_error",optimizer='adam', metrics=["mean_squared_error"])

    
    history=model.fit(np.array(trainX), np.array(trainy), validation_split=0.1,epochs=100, batch_size=2000, verbose=0)
    _,mse = model.evaluate(np.array(testX), np.array(testy),batch_size=1000,verbose=0)
    rmse = math.sqrt(abs(mse))
    df=pd.DataFrame({'Subject':[subject],
                     'Data point':[data_point],
                     'Step':[step],
                     'Question': [question],
                     'SignAl':[signal],
                    'RMSE':[rmse]})

    print("RMSE", str(rmse))
    return df
    
    
    
    

In [4]:
window=7000
output_length = 40
overlap = 100

data_points=[10] 
signal="EDA"
# signal_same_length=signal_same_length[8:]
# subjects = subjects[8:]
subject_no = 1
for subject  in subjects :
    print("----------------------- ***** ------------------")
    print("Subject: -> ", str(subject_no))
    for question in range(0,6):
        evaluation_list=[]
        y_new=[]
        y=[]
        with open('Dataset/QuestLabels/QuestLabels/'+subject+'.pickle', 'rb') as handle:
            b = pickle.load(handle)
            y.append(b['values']['Base']['stai'][question])
        y=pd.DataFrame(y)
        y[0]= y[0].map({1:0.25,2:0.5,3:0.75,4:1})
        y = list(y[0].values) 
        for x in range(0,7910): # copy same output value as target to all  signals for. here total generated overlap signal 7910
            y_new.append(y)
        print("Subject: ",subject)
        print("Question: ",question+1)
        X_new= build_data_with_forecasting(signal_same_length[subject_no-1],window,output_length, overlap)
        
        X=X_new.copy()
        y=y_new.copy()
        testX=X[7000:]
        testy=y[7000:]
        for data_point in data_points:
            for step in [1,2,3]:
                print("Question: ", str(question), end=",")
                print("Data Point: ", str(data_point), end=",")
                print("step: ", str(step))
                randomlist = random.sample(range(0, 7000), data_point) # Here 910 data points is for testing. thats why 7910-910=7000
                trainX = [X[i] for i in randomlist] 
                trainy = [y[i] for i in randomlist]
                eval_model = evaluate_model(trainX,trainy,testX,testy,subject,signal,question+1,data_point,step)
                evaluation_list.append(eval_model)
        df=pd.concat(evaluation_list)
        df.to_csv("Models/ClassificationForecasting_withoutPretrain_AndRandomDatapoints/"+signal+"/"+subject+"/MAE_Question_"+str(question)+".csv",index=False)
        print(f"df saved for {subject}, {question}")
    subject_no=subject_no+1
        
